In [7]:
import pandas as pd

In [8]:
df_dolly = pd.read_csv("../datasets/raw/dolly15k.csv")
df_opus = pd.read_csv("../datasets/raw/opus100-en-es.csv")
df_squad = pd.read_csv("../datasets/raw/squad_validation.csv")

In [3]:
import textclean

In [4]:
r = textclean.extract_col_model_metadata(df_dolly["context"], ["all-MiniLM-L6-v2"])

calculating embeddings for  context  with  all-MiniLM-L6-v2
Created embedding of shape (4466, 384)  with  all-MiniLM-L6-v2


In [ ]:
r = textclean.extract_col_model_metadata(df_opus["en"], ["all-mpnet-base-v2"])

In [ ]:
column_names = ["en", "es"]

df_opus["joint"] = df_opus[column_names].apply(lambda x: " ".join(x), axis=1)
df_opus

In [ ]:
column_names = ["en", "es"]
df_opus["joint"] = df_opus[column_names].apply(
    lambda x: " ".join([f"{col}: {val}" for col, val in zip(column_names, x)]), axis=1
)
df_opus

In [ ]:
df_opus["joint"].iloc[0]

In [ ]:
r

In [ ]:
opus_en_embeddings = textclean.calculate_embeddings(df_opus["en"], "all-mpnet-base-v2")

In [ ]:
import sentence_transformers

In [ ]:
avg_embed = opus_en_embeddings.mean(axis=0)
distances = sentence_transformers.util.cos_sim(avg_embed, opus_en_embeddings)

In [ ]:
distances.flatten().shape

In [ ]:
results = {}
results["en_embed_dist"] = distances.flatten()

In [ ]:
!pip install pyod

In [ ]:
from pyod.models.ecod import ECOD
from pyod.models.iforest import IForest

In [ ]:
opus_en_embeddings.shape

In [ ]:
def get_outlier_scores(data, model_name="ECOD"):
    if model_name == "ECOD":
        model = ECOD()
    else:
        model = IForest()
    model.fit(data)
    return model.decision_scores_


eocd_scores = get_outlier_scores(opus_en_embeddings, model_name="ECOD")
iforest_scores = get_outlier_scores(opus_en_embeddings, model_name="IForest")

In [ ]:
eocd_scores

In [ ]:
iforest_scores

# What can i do with embeddings to help with data cleaning?

1. Get outlier score for each point
2. Find near duplicates in embedding space
3. Given an individual point, find similar points


In [ ]:
from sentence_transformers import util

# cos_sim = util.cos_sim(emb1, emb2)

In [ ]:
df_opus["en_mean_embed_dist"] = distances.T

In [ ]:
df_opus

In [ ]:
distances = util.cos_sim(avg_en_embed, en_embeddings)

In [ ]:
distances

In [ ]:
avg_en_embed = en_embeddings.mean(axis=0)
distances = []

for i in range(en_embeddings.shape[0]):
    cos_sim = util.cos_sim(avg_en_embed, en_embeddings[i])
    distances.append(cos_sim)

In [ ]:
distances[:5]

In [ ]:
en_embeddings.mean(axis=0)

In [ ]:
avg_en = en_embeddings

In [ ]:
en_embeddings.shape

In [ ]:
# Our sentences we like to encode
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.',
#     'The quick brown fox jumps over the lazy dog.']

# Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(df_opus["en"])

# Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")